# Bidirectional LSTM on the IMDB sentiment classification task.
### Dr. Tirthajyoti Sarkar, Fremont, CA

IMDB dataset: https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

In [17]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.datasets import imdb

### How many features you will consider and how long embedding is to be used
Note, these choices will determine the size of the parameter space of your neural net.

In [2]:
max_features = 2000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100

### Load the data

Keras IMDB data sometimes gives trouble while loading. <br>
See this Stakoverflow thread for possible resolution,

https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56062555

In [3]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [4]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


### Build your model
Feel free to experiment with number of neurons and add more LSTM layers. But the irreducible error bar of the dataset can be reched pretty easily even with a simple network.

In [5]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Optimizer and compilation

In [6]:
opti_ = Adam(lr=0.0025)
model.compile(optimizer=opti_, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          256000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 354,945
Trainable params: 354,945
Non-trainable params: 0
_________________________________________________________________


### Fix a batch size, number of epochs to train, and off you go...

In [10]:
batch_size = 64
epochs = 20

In [11]:
from time import time

In [12]:
t1=time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[x_test, y_test])
t2=time()

Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 151s 6ms/step - loss: 0.4437 - acc: 0.7908 - val_loss: 0.3670 - val_acc: 0.8370
Epoch 2/20
25000/25000 [==============================] - 150s 6ms/step - loss: 0.3345 - acc: 0.8566 - val_loss: 0.3542 - val_acc: 0.8458
Epoch 3/20
25000/25000 [==============================] - 145s 6ms/step - loss: 0.3104 - acc: 0.8681 - val_loss: 0.3353 - val_acc: 0.8557
Epoch 4/20
25000/25000 [==============================] - 144s 6ms/step - loss: 0.2571 - acc: 0.8942 - val_loss: 0.3547 - val_acc: 0.8527
Epoch 5/20
25000/25000 [==============================] - 145s 6ms/step - loss: 0.2231 - acc: 0.9090 - val_loss: 0.3777 - val_acc: 0.8408
Epoch 6/20
25000/25000 [==============================] - 147s 6ms/step - loss: 0.1970 - acc: 0.9228 - val_loss: 0.3859 - val_acc: 0.8465
Epoch 7/20
25000/25000 [==============================] - 146s 6ms/step - 

In [20]:
t_delta = round((t2-t1)/60,3)
print(f"{epochs} took total {t_delta} minutes.")

20 took total 49.265 minutes.


### Some fancy things to try - `LearningRateScheduler` and `ReduceLROnPlateau` Callbacks
These are very useful callbacks to dynamically adjust the learning rate but they don't seem to impact the performance for this dataset.

In [86]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 0.005
    if epoch >= 3:
        lr *= 0.5
    if epoch >= 7:
        lr *= 0.25
    if epoch >= 11:
        lr *= 0.5
    if epoch >= 16:
        lr *= 0.5
        
    print('Learning rate: ', lr)
    return lr

In [87]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [88]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

In [90]:
callbacks = [lr_reducer, lr_scheduler]
#callbacks = [lr_scheduler]

In [91]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [92]:
opti_ = Adam(lr=lr_schedule(0))
model.compile(optimizer=opti_, loss='binary_crossentropy', metrics=['accuracy'])

Learning rate:  0.005


In [93]:
t1=time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[x_test, y_test],
          callbacks=callbacks)
t2=time()

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
Learning rate:  0.005
25000/25000 [==============================] - 152s 6ms/step - loss: 0.5266 - acc: 0.7340 - val_loss: 0.4281 - val_acc: 0.8036
Epoch 2/20
Learning rate:  0.005
25000/25000 [==============================] - 143s 6ms/step - loss: 0.3857 - acc: 0.8278 - val_loss: 0.3792 - val_acc: 0.8277
Epoch 3/20
Learning rate:  0.005
25000/25000 [==============================] - 145s 6ms/step - loss: 0.3255 - acc: 0.8610 - val_loss: 0.3572 - val_acc: 0.8411
Epoch 4/20
Learning rate:  0.0025
25000/25000 [==============================] - 143s 6ms/step - loss: 0.2665 - acc: 0.8888 - val_loss: 0.3616 - val_acc: 0.8477
Epoch 5/20
Learning rate:  0.0025
25000/25000 [==============================] - 145s 6ms/step - loss: 0.2340 - acc: 0.9050 - val_loss: 0.3770 - val_acc: 0.8463
Epoch 6/20
Learning rate:  0.0025
25000/25000 [==============================] - 146s 6ms/step - loss: 0.2087 - acc: 0.9166 - val_loss: 0.4063 - val